# plan

ideas from literature:

1. stanford n-gram generator. they used tf-icf, which is particularly tuned to the input lyrics. at the first stage we don't take input lyrics into account. Also: syllable rhyming resource.

2. an interesting idea is the Japanese system where they used topic models to sample words, so they can cater to different topics.

# Rhyming

In [143]:
import pronouncing
def is_rhyme(word1,word2):
    return word1 in pronouncing.rhymes(word2)
print is_rhyme('hood','good')
print is_rhyme('hood','hooded')

True
False


True

# to-do: implement linear-interpolation in ngram SRILM

principled way to do this:

1. we already have a dev set and train set. we can train LM on train set, vary:

-addsmooth

-linear interpolation

2. then we can test on the dev set to get the lowest perplexity.

3. we need to clean the text data by removing any non-alphanumeric characters and by removing the song meta line.

# [5/1]you need to first train a unigram model, a bigram model, then coupled with your trigram model. we need to clean the text data by removing any non-alphanumeric characters and by removing the song meta line.

# [5/1]the lesson today is that perplexity doesn't seem to always work right now. the lowest ppl is a good sentence, but the next bests are not that good. we need either a better LM or to generate better sentences, because grammaticality is low now. I am counting on a better LM to solve this, maybe the linear interpolation. also try fourgram. 

# [5/2]we've got it to work: train LM using srilm, and use the kenlm python 3 interface to load the LM and score sentences. after comparing the scoring in python terminal (see my dump txt file of the terminal output), it seems like high score (loglik) corresponds to shorter sentences, which makes sense, although there are some longer sents that are good, they don't necessarily have the best score. We should normalize by sent length to derive the norm_score. Then it should be good. Check stanford paper to see what other tweaks they did.

# done cleaning corpus: next: (1) train linear interpolation LM; (2) we are ready to do feature extraction.

In [ ]:
#assign perplexity to dev set data

# ML: classify a line into whether its a good next line (=1) or not(=0)

i. How to rank?

1) We train a classifier to rank them. Specifically, the classifier will give the a bunch of features based on current candidates and previous line, and it will classify each candidate as a good next line or not (binary).

ii. Features:

1) Perplexity(correct syntax)

2) Coherence (semantic relatedness)

- D2V cosine distance b/t current candidate and previous line

- W2V average distance b/t current cand and prev line

- Other measures from CompDisc

3) Other song-specific constratraints 

- Rhyme (with prev line, or within the cand line)

- Syllables (length: number of syllable of the current cand and prev line)

- ?>Timing<?

4) Training the classifier: we will use distant supervision - where we use these two kinds of training data:

- Next_line: for each line, compute features for its next line as above, class label is 1;

- Not_next_line: for each line, randomly sample a line from anywhere outside of the song, and compute their features, class label is 0;

- we need to also include some automatically generated lines with low log likelihood


# clean corpus

In [ ]:
#1. get rid of metadata, also ()

In [41]:
ls

README.md         build_data/       gen_lyrics.ipynb


In [42]:
ls build_data/


build_dataset.ipynb    lyrics_test_data.txt   lyrics_train_data.txt


In [138]:
import re 

train_data='build_data/lyrics_train_data.txt'
test_data='build_data/lyrics_test_data.txt'
train_data_clean=train_data.replace('.txt','_clean.txt')
test_data_clean=test_data.replace('.txt','_clean.txt')
#data_str=open(data,'r').read()
def clean_data(data_str):
    data_str=data_str.replace('(','')
    data_str=data_str.replace(')','')
    data_str=data_str.replace('*CHORUS*','')
    data_str=data_str.replace('*Chorus:','')
    
    
    
    pat=u'SONG_META_ARTIST_TITLE:.+\n'
    patt=u'\[.*\]'
    cleaned=re.sub(pat,'',data_str)
    cleaned=re.sub(patt,'',cleaned)
    cleaned=re.sub(re.compile(u'^verse.*\n',re.MULTILINE|re.I),'',cleaned)
    cleaned=re.sub(re.compile(u'^\*',re.MULTILINE),'',cleaned)
    cleaned=re.sub(re.compile(u'\*\n'),'\n',cleaned)
    cleaned=re.sub(re.compile(u'^\S+:\s*\n',re.MULTILINE),'\n',cleaned)
    cleaned=re.sub(re.compile(u'^[Rr]epeat.*\n',re.MULTILINE),'',cleaned)
    cleaned=re.sub(re.compile(u'^chorus.*\n',re.MULTILINE|re.I),'',cleaned)
    
    return cleaned
    
def clean_file(fn,output):
    data_str=open(fn,'r').read()
    cleaned=clean_data(data_str)
    f=open(output,'w')
    f.write(cleaned)
    
    

In [129]:
a="""repeat

*repeat coh

repeat chorus

Repeat dgh

*cladgn*

dagsadg

chorus

Verse:
adsgasdga asdgasdg asdgag
asdgasdg asgasdg


soemthing:


Chorus:

CHORUS:

something"""

a="""

Verse: 
Hiding in the television again 
Looking for anyone to turn you on 
Caught on film by a stranger's eye 
Just another place I don't belong """

#print re.sub(re.compile(u'^[Rr]epeat.*\n',re.MULTILINE),'',a)
a=re.sub(re.compile(u'^\*',re.MULTILINE),'',a)    
a=re.sub(re.compile(u'\*\n',re.MULTILINE),'\n',a)
a=re.sub(re.compile(u'^[Rr]epeat.*\n',re.MULTILINE),'',a)
a=re.sub(re.compile(u'^\S+:\n'),'\n',a)
print re.sub(re.compile(u'^chorus.*\n',re.MULTILINE|re.I),'',a)



Verse: 
Hiding in the television again 
Looking for anyone to turn you on 
Caught on film by a stranger's eye 
Just another place I don't belong 


In [133]:
m=re.search(re.compile(u'^\S+:\s*\n',re.MULTILINE),a)
print m.group()

Verse: 



In [117]:
b="""
Letters still unopened in a home that's not a home 
Ain't nothing going to ever change that 
The writings on the wall in words I never wrote 
Could it be my lies after all 

PreChorus: 
Staring in the mirror at these patterns of odd 
Taking after your mother another should have been 
Ending up like superman in the closet 
Unable to come out 

Just one more game where someone has to lose 
Just one more way to separate the truth 
Just one more line they already used 
Just one more me 
Pretending to be you 

Verse: 
Hiding in the television again 
Looking for anyone to turn you on 
Caught on film by a stranger's eye 
Just another place I don't belong 

PreChorus: 
Breaking mirrors of there patterns of odd 
Blaming your father for who you really are 
Placing ads for heroes cause you can't save yourself 

Just one more game where someone has to lose 
Just one more way to separate the truth 
Just one more line they already used 
Just one more me 
Pretending to be you
"""

In [134]:
a=b
a=re.sub(re.compile(u'^\*',re.MULTILINE),'',a)    
a=re.sub(re.compile(u'\*\n',re.MULTILINE),'\n',a)
a=re.sub(re.compile(u'^[Rr]epeat.*\n',re.MULTILINE),'',a)
a=re.sub(re.compile(u'^\S+:\s*\n',re.MULTILINE),'\n',a)
print re.sub(re.compile(u'^chorus.*\n',re.MULTILINE|re.I),'',a)


Letters still unopened in a home that's not a home 
Ain't nothing going to ever change that 
The writings on the wall in words I never wrote 
Could it be my lies after all 


Staring in the mirror at these patterns of odd 
Taking after your mother another should have been 
Ending up like superman in the closet 
Unable to come out 

Just one more game where someone has to lose 
Just one more way to separate the truth 
Just one more line they already used 
Just one more me 
Pretending to be you 


Hiding in the television again 
Looking for anyone to turn you on 
Caught on film by a stranger's eye 
Just another place I don't belong 


Breaking mirrors of there patterns of odd 
Blaming your father for who you really are 
Placing ads for heroes cause you can't save yourself 

Just one more game where someone has to lose 
Just one more way to separate the truth 
Just one more line they already used 
Just one more me 
Pretending to be you



## verse vs chorus

there are about 270 chrous passages in the entire training + test data, not enough to train a separate LM. 

However, is it possible we can extract some features/contraints from these data?

actually, we shouldn't, because there are many chorus lines in the corpus that is not marked as chorus (most of them), so therefore this training division doesn't make sense. For now we'll remove these chorus markers, but in the future ideally we can build and train two data sets on verse and chorus.

## clean anything thats within a bracket like [...]

In [72]:
test_bra="""In the beginning was a golden ray
God's power, but to my dismay
"We're only human," that's what people say
Still I search far away

[Chorus]

There but for the grace of God go I
I pretend I'm not afraid to die
If there's a heaven, then I will apply
Devils heat, you better take a ride
I'm not crazy, but I believe in butterflies
Dreams that die, should be a big surprise
And life is my favourite nursery rhyme
I'm not afraid of the big bad wolf

[Chorus]



Oh, so strong
Tell me stories of distant shores
All night long
I shiver all over
When I see your lovly tan
And I can tell by your clear blue eyes
You're a sailor man

Na na... na na... na na... na na... na na na [3x]
Oh, so cruel to me.

Oh, why has it taken so long
To see what's been happening to me?
This time around, hey, I'm gone.
Hey, I'm gone.

You better get ready for a stormy day
'cuz everything you love's about to wash away.
I'm not gonna stand around and let you be
So cruel to me. No.

I'm not gonna stand around and let you be 
So cruel to me. No.

Na na... na na... na na... na na... na na na [3x]

Not gonna stand around and let you be"""


In [75]:
test=test+test_bra

In [76]:

print clean_data(test)


But don't leave your keys for me, open the door
Nobody's getting hurt here if nobody cares
Nobody's getting lost if nobody's going anywhere

And darling I know that you can't be sure of anything anymore

I wanna love you like it ain't no secret
Like I'm not ashamed to show
Nor would I ever, oh never never
Oh never let you go
I'll never let you go

I'm on a tight rope maybe I'm scared
Maybe I love you but you never play fair
So shed a little light where nobody sees
I built a bible for the saints with my disbelief

And darling I know that you can't be sure of anything anymore

I wanna love you like it ain't no secret

Like I'm not ashamed to show
Nor would I ever, oh never never
Oh never let you go

My love is easy
Tell me bout yours and then
Tell me you need me
Carry me home again
It's been a mystery
Most of the time
Stranded on the big top
Throw me a line

I wanna love you like it ain't no secret
Like I'm not ashamed to show
Nor would I ever, oh never never
Oh never let you go
I'll ne

In [43]:
test="""SONG_META_ARTIST_TITLE:The Alternate Routes   Ain't No Secret

But don't leave your keys for me, open the door
Nobody's getting hurt here if nobody cares
Nobody's getting lost if nobody's going anywhere

And darling I know that you can't be sure of anything anymore

I wanna love you like it ain't no secret
Like I'm not ashamed to show
Nor would I ever, oh never never
Oh never let you go
I'll never let you go

I'm on a tight rope maybe I'm scared
Maybe I love you but you never play fair
So shed a little light where nobody sees
I built a bible for the saints with my disbelief

And darling I know that you can't be sure of anything anymore

I wanna love you like it ain't no secret

Like I'm not ashamed to show
Nor would I ever, oh never never
Oh never let you go

My love is easy
Tell me bout yours and then
Tell me you need me
Carry me home again
It's been a mystery
Most of the time
Stranded on the big top
Throw me a line

I wanna love you like it ain't no secret
Like I'm not ashamed to show
Nor would I ever, oh never never
Oh never let you go
I'll never let you go


SONG_META_ARTIST_TITLE:Kurt Nilsen   Come On

It's my turn, searching for another break
Set me free, I only see the setting sun
Let me feel, like my luck has just begun
Wherever I go, it's all the same
We're so much alike, well, with a different name
Days going by, I'm gonna catch 'em again
I'm gonna lose this half-life, I'm on my way

Come on, we never know before we try
Come on, theres nothing we can't leave behind
Come on, we're past the time for questions, why?
Come on, yeah come on, what are we waiting for
Come on

Looking back, what happened to the time we spent
It's gone, no small cost for a life of rent
Let it go, Regret are just for feeling low
Here and now, the only thing that's left to know

Come on, we never know before we try
Come on, theres nothing we can't leave behind
Come on, we're past the time for questions, why
Come on, yeah come on, what are we waiting for

We owe it to ourselves now
And no one else can live our dreams
We owe it to ourselves, come on, come on
What are we waiting for
Come on, we never know before we try
Come on, there's nothing we can't leave behind
Come on, we're past the time for questions, why
Come on, yeah come on, what are we waiting for
Uhhhh, come on,
Come on, we never know before we try
So, come on, come on what are we waiting for
Get it on, It's my turn, set me free
Why don't you set me free, let me feel
Uh, yeah, yeah, yeah
Bring it on, bring it on, It's my turn
Set me free
Talk to me throw the ball back
Let me feel, come on


SONG_META_ARTIST_TITLE:Lemonator   Over Now

You love me
You love me like you love no one
You leave me
You leave me
You leave me for the second time
You come back
You come back
You come back like you did before
Youre wasted
Youre wasted
Youre wasted like you were in

California, fairyland
California, you can't look bad
Were over
Were over

Were over with this you and me
Youre happy
Im happy
Were happy in our make belive
Youre with another one
Youre with another one

Youre with that neighbours deep tanned blonde
You scumbag
You scumbag

Youre scum and you can go to
California, fairyland
California, you can't look bad
California, fairyland
California, you can't look bad
No, no you can't be like that
No way, I can't be like that
Oh no, well soon be like that
California, fairyland
California, you can't look bad
California, fairyland
California, you can't look bad


SONG_META_ARTIST_TITLE:Alison Moyet   Find Me

That it weighs you down
So all you have is time
Don't turn away
We have both seen darker days
And there's always a light that shines

(Do you feel like) all your hopes are a long way gone
(Does it seem like) every road only takes you down
(Do they shut you out)
Don't be afraid of what they say
Throw back your head and cry

When you need somebody, find me there
When you're feeling empty I'll be there - find me

When you're feeling weary with no peace of mind
And the tone of your voice is hollow
You're home to stranger that won't belong
And there's no-one else to follow

(Do you feel like) nowhere, oh no, there's no better way
(Does it seem like) nobody hears a word you say
(And they shut you out)
And where does it say that a man aint a right
To throw back his head and cry

You don't need to tell me 'cos I know
I know how you feel (you're miles away)
You don't need to show me (nobody listens to a word you say)
We can shut them out
Where does it say that a man aint a right
To throw back his head and cry

When you need somebody find me there
I'll be right beside you
When you're feeling empty I'll be there
Can you hear me? I'll be singing
"""

## data cleaning: putting it in action

In [140]:
clean_file(test_data,test_data_clean)
clean_file(train_data,train_data_clean)

In [116]:
pwd

u'/Users/zangsir/repo/songLyrics'

## scramble


In [56]:
import re
pat=u'SONG_META_ARTIST_TITLE:.+\n'
#match=re.search(pat,test)
new_test=re.sub(pat,'',test)

In [51]:
match.group()

"SONG_META_ARTIST_TITLE:The Alternate Routes   Ain't No Secret\n"

In [53]:
new_test=re.sub(pat,'',test)

In [55]:
print new_test


But don't leave your keys for me, open the door
Nobody's getting hurt here if nobody cares
Nobody's getting lost if nobody's going anywhere

And darling I know that you can't be sure of anything anymore

I wanna love you like it ain't no secret
Like I'm not ashamed to show
Nor would I ever, oh never never
Oh never let you go
I'll never let you go

I'm on a tight rope maybe I'm scared
Maybe I love you but you never play fair
So shed a little light where nobody sees
I built a bible for the saints with my disbelief

And darling I know that you can't be sure of anything anymore

I wanna love you like it ain't no secret

Like I'm not ashamed to show
Nor would I ever, oh never never
Oh never let you go

My love is easy
Tell me bout yours and then
Tell me you need me
Carry me home again
It's been a mystery
Most of the time
Stranded on the big top
Throw me a line

I wanna love you like it ain't no secret
Like I'm not ashamed to show
Nor would I ever, oh never never
Oh never let you go
I'll ne